<a href="https://colab.research.google.com/github/cesaraugf/py-sql-github-data-project/blob/develop/enriquecimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [3]:
df=con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10004,SERRA,BT50,100,200,z0019_2.csv,2025-10-15 22:00:03.870955
1,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-10-15 22:00:03.870955
2,10003,PREGO,BT10,100,60,z0019_2.csv,2025-10-15 22:00:03.870955
3,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-10-15 22:00:43.164246
4,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-10-15 22:00:43.164246
5,10003,PREGO,BT10,100,50,z0019_1.csv,2025-10-15 22:00:43.164246


In [4]:
df = con.execute("""
                SELECT * 
                FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) as ROW
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-10-15'
                 ) WHERE ROW = 1
                """).fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,ROW
0,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-10-15 22:00:43.164246,1
1,10003,PREGO,BT10,100,50,z0019_1.csv,2025-10-15 22:00:43.164246,1
2,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-10-15 22:00:43.164246,1
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-10-15 22:00:03.870955,1
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-10-15 22:00:03.870955,1


In [7]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'ROW'])
df_final = df_final.rename(columns={'NATBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500
1,10003,PREGO,BT10,100,50
2,10001,PARAFUSO,BT10,100,100
3,10004,SERRA,BT50,100,200
4,10005,MACHADO,BT50,100,100


In [8]:
con.commit()